In [90]:
import numpy as np
import pandas as pd


# Files to Load
wsb1_data_to_load = "data/WSB_rd1_count.csv"
wsb2_data_to_load = "data/WSB_rd2_count.csv"
wsb3_data_to_load = "data/WSB_rd3_count.csv"

In [91]:
# Read the WSB_Reddit Data from 3 Saved Dataframes Files from 3 Batches of Reddit api Called
wsb_rd1 = pd.read_csv(wsb1_data_to_load)
wsb_rd2 = pd.read_csv(wsb2_data_to_load)
wsb_rd3 = pd.read_csv(wsb3_data_to_load)

- Vertical merge the above three dataframes, wsb_rd1, wsb_rd2 and wsb_rd3, for a WSB_reddits dataframe as a preparation for nlp analysis.
- Sort the dataframe in descending date order for a purpose of web display which allows about 125,000 rows so that the most recent rows meet the cut.

In [92]:
frames = [wsb_rd1, wsb_rd2, wsb_rd3]
WSB_reddits = pd.concat(frames)
WSB_reddits['date'] = pd.to_datetime(WSB_reddits.date)
WSB_reddits.sort_values('date', inplace=True)
WSB_reddits = WSB_reddits.loc[WSB_reddits['date']>'2016-01-01']
WSB_reddits = WSB_reddits.loc[WSB_reddits['text']!='[removed]']
WSB_reddits = WSB_reddits.loc[WSB_reddits['text']!='[deleted]']
WSB_reddits = WSB_reddits.loc[WSB_reddits['text'].notna()]
WSB_reddits = WSB_reddits.loc[:, ["date", "text", "favorites" ]].dropna()
WSB_reddits.sort_values(by=['date'], inplace=True, ascending=False)

WSB_reddits.head()

,date,text,favorites
583031,2021-05-01 00:47:49,$GTEH sells sinfit protein enhanced foods and ...,1
583030,2021-05-01 00:44:06,BUY ALERT! WSGF... World Series of Golfs Subsi...,1
583026,2021-05-01 00:39:23,We can keep things fresh for all with SCYX Vag...,1
583016,2021-05-01 00:21:36,I like MVIS and personally think it will rise ...,1
583002,2021-05-01 00:12:15,there is a poocoin telegram group that recentl...,1


In [93]:
WSB_reddits.to_csv('./Data/WSB_rd_count.csv', index=False)

In [94]:
reddits = WSB_reddits.text

In [95]:
# Instantiate Count Vectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 

my_additional_stop_words = ['https','rt', 'amp', 'com']

stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
vectorizer = CountVectorizer( lowercase=True, stop_words=stop_words)

vector_text = reddits

# To actually create the vectorizer, we simply need to call fit on the text data that we wish to fix
cv_fit = vectorizer.fit_transform(vector_text)

In [96]:
words = vectorizer.vocabulary_

In [97]:
feature_names = vectorizer.get_feature_names()
feature_counts = cv_fit.sum(axis=0).A1

In [98]:
result=dict(zip(feature_names, feature_counts))
result

{'00': 7980,
 '000': 16245,
 '0000': 41,
 '00000': 98,
 '000000': 13,
 '0000000': 1,
 '00000000': 4,
 '000000000': 1,
 '000000000000000001': 1,
 '000000000000002': 1,
 '00000000009': 1,
 '0000000001': 1,
 '000000001': 1,
 '000000003880': 1,
 '000000005360924': 1,
 '00000001': 4,
 '00000003': 1,
 '00000006': 1,
 '000000093': 1,
 '0000001': 4,
 '00000011': 1,
 '000000248817000107': 1,
 '000000248818000102': 1,
 '000000248818000172': 2,
 '000000248819000090': 1,
 '00000024956': 1,
 '0000003402': 1,
 '000000496217000071': 1,
 '000000595': 1,
 '000000620120000066': 1,
 '000000857': 2,
 '000001': 15,
 '0000010011101110000001101010111111110001011101100001001111010100111000000100001110100111100101101110010011010101010011010110011001': 1,
 '00000142': 1,
 '0000016387': 1,
 '0000019': 1,
 '000001961720000191': 1,
 '000001961720000513': 1,
 '000002': 3,
 '0000032604': 1,
 '0000054': 1,
 '000006': 2,
 '000007': 2,
 '000007632119000082': 2,
 '000007800320000021': 1,
 '000007995815000035': 2,
 '0000

In [99]:
final_count_df = pd.DataFrame({"word": feature_names, "cnt_word": feature_counts})
final_count_df.sort_values(by='cnt_word', ascending=False)

,word,cnt_word
173362,just,80484
240827,stock,79454
186557,market,75627
181265,like,73419
89386,buy,63106
...,...,...
147841,gbmzxs,1
147842,gbn13d,1
147843,gbn400,1
147844,gbn5we,1


In [100]:
# Introduce adjusted economic keywords extracted from top 100 words in the combolist in Powell's speeches + keywords in economic indicators
trade_matchers = ["bankruptcies",
"broadband internet penetration",
"building permit",
"business cycle",
"central banks",
"changes in",
"china",
"construction spending",
"consumer confidence index",
"consumer leverage ratio",
"consumer price index",
"consumer spending",
"during the",
"economic growth",
"employment",
"export",
"federal funds",
"federal open",
"federal reserve",
"financial crisis",
"financial stability",
"financial system",
"forward to",
"funds rate",
"gdp",
"home building",
"housing starts",
"import",
"importance of",
"industrial production",
"inflation",
"interest rate spread",
"interest rates",
"jobless claims",
"labor market",
"level of",
"likely to",
"longer term",
"look forward",
"manufacture new order",
"manufacture",
"manufacturing demand",
"manufacturing",
"market committee",
"market",
"maximum employment",
"monetary policy",
"money supply",
"money",
"monthly job added",
"NAFTA",
"NASDAQ",
"need to",
"new residential sales",
"oil",
"open market",
"opportunity to",
"our percent",
"over time",
"percent objective",
"prime rate",
"producer price index",
"quarterly change in gdp",
"retail sales",
"review of",
"S&P 500",
"short term",
"stock market prices",
"stock market",
"stocks",
"tariff",
"tax",
"the committee",
"the economy",
"the fed",
"the federal",
"the fomc",
"the great",
"the outlook",
"the united",
"the world",
"to address",
"to continue",
"to ensure",
"to help",
"to make",
"to provide",
"to speak",
"to support",
"trade",
"turn to",
"understanding of",
"unemployment",
"united states",
"vendor performance",
"weekly hours",
"wholesale price index"]

In [101]:
import datetime
from collections import defaultdict
import numpy as np

In [102]:
WSB_reddits['week'] = (WSB_reddits.date - WSB_reddits['date'].dt.weekday * np.timedelta64(1,'D'))
WSB_reddits['week'] = pd.DatetimeIndex(WSB_reddits['week']).normalize()
WSB_reddits['day'] = pd.DatetimeIndex(WSB_reddits['date']).normalize()
WSB_reddits

,date,text,favorites,week,day
583031,2021-05-01 00:47:49,$GTEH sells sinfit protein enhanced foods and ...,1,2021-04-26,2021-05-01
583030,2021-05-01 00:44:06,BUY ALERT! WSGF... World Series of Golfs Subsi...,1,2021-04-26,2021-05-01
583026,2021-05-01 00:39:23,We can keep things fresh for all with SCYX Vag...,1,2021-04-26,2021-05-01
583016,2021-05-01 00:21:36,I like MVIS and personally think it will rise ...,1,2021-04-26,2021-05-01
583002,2021-05-01 00:12:15,there is a poocoin telegram group that recentl...,1,2021-04-26,2021-05-01
...,...,...,...,...,...
34,2016-01-01 21:10:15,Twice as much gainz over the last 10 years. T...,1,2015-12-28,2016-01-01
33,2016-01-01 18:58:08,I noticed they bought out optionsexpress Canad...,1,2015-12-28,2016-01-01
32,2016-01-01 18:02:39,They have recently acquiered SNDK and their on...,0,2015-12-28,2016-01-01
28,2016-01-01 03:41:28,I believe stock has potential for a break out ...,8,2015-12-28,2016-01-01


In [103]:
# Count frequencies of economic keywords by week
countbyweek=defaultdict(int)
for index,row in WSB_reddits.iterrows():
    for word in range(len(row['text'].split(' '))-1):
        words = row['text'].split(' ')
        if f'{words[word]} {words[word +1]}' in trade_matchers:
            countbyweek[row['week']] += 1
#     break
countbyweek

defaultdict(int,
            {Timestamp('2021-04-26 00:00:00'): 591,
             Timestamp('2021-04-19 00:00:00'): 559,
             Timestamp('2021-04-12 00:00:00'): 514,
             Timestamp('2021-04-05 00:00:00'): 720,
             Timestamp('2021-03-29 00:00:00'): 965,
             Timestamp('2021-03-22 00:00:00'): 76,
             Timestamp('2021-03-15 00:00:00'): 802,
             Timestamp('2021-03-08 00:00:00'): 2351,
             Timestamp('2021-03-01 00:00:00'): 688,
             Timestamp('2021-02-22 00:00:00'): 842,
             Timestamp('2021-02-15 00:00:00'): 1345,
             Timestamp('2021-02-08 00:00:00'): 904,
             Timestamp('2021-02-01 00:00:00'): 1871,
             Timestamp('2021-01-25 00:00:00'): 6292,
             Timestamp('2021-01-18 00:00:00'): 1243,
             Timestamp('2021-01-11 00:00:00'): 726,
             Timestamp('2021-01-04 00:00:00'): 483,
             Timestamp('2020-12-28 00:00:00'): 527,
             Timestamp('2020-12-21 00:00:00

In [104]:
# Count frequencies of economic keywords by day
countbyday=defaultdict(int)
for index,row in WSB_reddits.iterrows():
    for word in range(len(row['text'].split(' '))-1):
        words = row['text'].split(' ')
        if f'{words[word]} {words[word +1]}' in trade_matchers:
            countbyday[row['day']] += 1
#     break
countbyday

defaultdict(int,
            {Timestamp('2021-05-01 00:00:00'): 1,
             Timestamp('2021-04-30 00:00:00'): 148,
             Timestamp('2021-04-29 00:00:00'): 84,
             Timestamp('2021-04-28 00:00:00'): 124,
             Timestamp('2021-04-27 00:00:00'): 144,
             Timestamp('2021-04-26 00:00:00'): 90,
             Timestamp('2021-04-25 00:00:00'): 55,
             Timestamp('2021-04-24 00:00:00'): 85,
             Timestamp('2021-04-23 00:00:00'): 83,
             Timestamp('2021-04-22 00:00:00'): 54,
             Timestamp('2021-04-21 00:00:00'): 80,
             Timestamp('2021-04-20 00:00:00'): 83,
             Timestamp('2021-04-19 00:00:00'): 119,
             Timestamp('2021-04-18 00:00:00'): 65,
             Timestamp('2021-04-17 00:00:00'): 127,
             Timestamp('2021-04-16 00:00:00'): 145,
             Timestamp('2021-04-15 00:00:00'): 111,
             Timestamp('2021-04-14 00:00:00'): 66,
             Timestamp('2021-04-09 00:00:00'): 138,
       

In [105]:
# Create a dataframe of the frequency counts of economic keywords by week
wsb_weekdf = pd.DataFrame([countbyweek]).transpose().reset_index()
wsb_weekdf.columns=(['Week','Count'])
wsb_weekdf

,Week,Count
0,2021-04-26,591
1,2021-04-19,559
2,2021-04-12,514
3,2021-04-05,720
4,2021-03-29,965
...,...,...
274,2016-01-25,37
275,2016-01-18,55
276,2016-01-11,48
277,2016-01-04,39


In [106]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models 
wsb_weekdf.to_csv('./Data/wsb_weekdf_count.csv', index=False)

In [107]:
# Create a dataframe of the frequency counts of economic keywords by day
wsb_daydf = pd.DataFrame([countbyday]).transpose().reset_index()
wsb_daydf.columns=(['Date','Count'])
wsb_daydf

,Date,Count
0,2021-05-01,1
1,2021-04-30,148
2,2021-04-29,84
3,2021-04-28,124
4,2021-04-27,144
...,...,...
1862,2016-01-06,9
1863,2016-01-05,4
1864,2016-01-04,9
1865,2016-01-03,11


In [108]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models 
wsb_daydf.to_csv('./Data/wsb_daydf_count.csv', index=False)

In [111]:
# Introduce negative financial keywords
Negative = ['abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonments', 'abandons', 'abdicated',
                       'abdicates', 'abdicating', 'abdication', 'abdications', 'aberrant', 'aberration', 'aberrational',
                       'aberrations', 'abetting', 'abnormal', 'abnormalities', 'abnormality', 'abnormally', 'abolish',
                       'abolished', 'abolishes', 'abolishing', 'abrogate', 'abrogated', 'abrogates', 'abrogating',
                       'abrogation', 'abrogations', 'abrupt', 'abruptly', 'abruptness', 'absence', 'absences',
                       'absenteeism', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abusively', 'abusiveness',
                       'accident', 'accidental', 'accidentally', 'accidents', 'accusation', 'accusations', 'accuse',
                       'accused', 'accuses', 'accusing', 'acquiesce', 'acquiesced', 'acquiesces', 'acquiescing',
                       'acquit', 'acquits', 'acquittal', 'acquittals', 'acquitted', 'acquitting', 'adulterate',
                       'adulterated', 'adulterating', 'adulteration', 'adulterations', 'adversarial', 'adversaries',
                       'adversary', 'adverse', 'adversely', 'adversities', 'adversity', 'aftermath', 'aftermaths',
                       'against', 'aggravate', 'aggravated', 'aggravates', 'aggravating', 'aggravation', 'aggravations',
                       'alerted', 'alerting', 'alienate', 'alienated', 'alienates', 'alienating', 'alienation',
                       'alienations', 'allegation', 'allegations', 'allege', 'alleged', 'allegedly', 'alleges',
                       'alleging', 'annoy', 'annoyance', 'annoyances', 'annoyed', 'annoying', 'annoys', 'annul',
                       'annulled', 'annulling', 'annulment', 'annulments', 'annuls', 'anomalies', 'anomalous',
                       'anomalously', 'anomaly', 'anticompetitive', 'antitrust', 'argue', 'argued', 'arguing',
                       'argument', 'argumentative', 'arguments', 'arrearage', 'arrearages', 'arrears', 'arrest',
                       'arrested', 'arrests', 'artificially', 'assault', 'assaulted', 'assaulting', 'assaults',
                       'assertions', 'attrition', 'aversely', 'backdating', 'bad', 'bail', 'bailout', 'balk', 'balked',
                       'bankrupt', 'bankruptcies', 'bankruptcy', 'bankrupted', 'bankrupting', 'bankrupts', 'bans',
                       'barred', 'barrier', 'barriers', 'bottleneck', 'bottlenecks', 'boycott', 'boycotted',
                       'boycotting', 'boycotts', 'breach', 'breached', 'breaches', 'breaching', 'break', 'breakage',
                       'breakages', 'breakdown', 'breakdowns', 'breaking', 'breaks', 'bribe', 'bribed', 'briberies',
                       'bribery', 'bribes', 'bribing', 'bridge', 'broken', 'burden', 'burdened', 'burdening', 'burdens',
                       'burdensome', 'burned', 'calamities', 'calamitous', 'calamity', 'cancel', 'canceled',
                       'canceling', 'cancellation', 'cancellations', 'cancelled', 'cancelling', 'cancels', 'careless',
                       'carelessly', 'carelessness', 'catastrophe', 'catastrophes', 'catastrophic', 'catastrophically',
                       'caution', 'cautionary', 'cautioned', 'cautioning', 'cautions', 'cease', 'ceased', 'ceases',
                       'ceasing', 'censure', 'censured', 'censures', 'censuring', 'challenge', 'challenged',
                       'challenges', 'challenging', 'chargeoffs', 'circumvent', 'circumvented', 'circumventing',
                       'circumvention', 'circumventions', 'circumvents', 'claiming', 'claims', 'clawback', 'closed',
                       'closeout', 'closeouts', 'closing', 'closings', 'closure', 'closures', 'coerce', 'coerced',
                       'coerces', 'coercing', 'coercion', 'coercive', 'collapse', 'collapsed', 'collapses',
                       'collapsing', 'collision', 'collisions', 'collude', 'colluded', 'colludes', 'colluding',
                       'collusion', 'collusions', 'collusive', 'complain', 'complained', 'complaining', 'complains',
                       'complaint', 'complaints', 'complicate', 'complicated', 'complicates', 'complicating',
                       'complication', 'complications', 'compulsion', 'concealed', 'concealing', 'concede', 'conceded',
                       'concedes', 'conceding', 'concern', 'concerned', 'concerns', 'conciliating', 'conciliation',
                       'conciliations', 'condemn', 'condemnation', 'condemnations', 'condemned', 'condemning',
                       'condemns', 'condone', 'condoned', 'confess', 'confessed', 'confesses', 'confessing',
                       'confession', 'confine', 'confined', 'confinement', 'confinements', 'confines', 'confining',
                       'confiscate', 'confiscated', 'confiscates', 'confiscating', 'confiscation', 'confiscations',
                       'conflict', 'conflicted', 'conflicting', 'conflicts', 'confront', 'confrontation',
                       'confrontational', 'confrontations', 'confronted', 'confronting', 'confronts', 'confuse',
                       'confused', 'confuses', 'confusing', 'confusingly', 'confusion', 'conspiracies', 'conspiracy',
                       'conspirator', 'conspiratorial', 'conspirators', 'conspire', 'conspired', 'conspires',
                       'conspiring', 'contempt', 'contend', 'contended', 'contending', 'contends', 'contention',
                       'contentions', 'contentious', 'contentiously', 'contested', 'contesting', 'contraction',
                       'contractions', 'contradict', 'contradicted', 'contradicting', 'contradiction', 'contradictions',
                       'contradictory', 'contradicts', 'contrary', 'controversial', 'controversies', 'controversy',
                       'convict', 'convicted', 'convicting', 'conviction', 'convictions', 'corrected', 'correcting',
                       'correction', 'corrections', 'corrects', 'corrupt', 'corrupted', 'corrupting', 'corruption',
                       'corruptions', 'corruptly', 'corruptness', 'costly', 'counterclaim', 'counterclaimed',
                       'counterclaiming', 'counterclaims', 'counterfeit', 'counterfeited', 'counterfeiter',
                       'counterfeiters', 'counterfeiting', 'counterfeits', 'countermeasure', 'countermeasures', 'crime',
                       'crimes', 'criminal', 'criminally', 'criminals', 'crises', 'crisis', 'critical', 'critically',
                       'criticism', 'criticisms', 'criticize', 'criticized', 'criticizes', 'criticizing', 'crucial',
                       'crucially', 'culpability', 'culpable', 'culpably', 'cumbersome', 'curtail', 'curtailed',
                       'curtailing', 'curtailment', 'curtailments', 'curtails', 'cut', 'cutback', 'cutbacks',
                       'cyberattack', 'cyberattacks', 'cyberbullying', 'cybercrime', 'cybercrimes', 'cybercriminal',
                       'cybercriminals', 'damage', 'damaged', 'damages', 'damaging', 'dampen', 'dampened', 'danger',
                       'dangerous', 'dangerously', 'dangers', 'deadlock', 'deadlocked', 'deadlocking', 'deadlocks',
                       'deadweight', 'deadweights', 'debarment', 'debarments', 'debarred', 'deceased', 'deceit',
                       'deceitful', 'deceitfulness', 'deceive', 'deceived', 'deceives', 'deceiving', 'deception',
                       'deceptions', 'deceptive', 'deceptively', 'decline', 'declined', 'declines', 'declining',
                       'deface', 'defaced', 'defacement', 'defamation', 'defamations', 'defamatory', 'defame',
                       'defamed', 'defames', 'defaming', 'default', 'defaulted', 'defaulting', 'defaults', 'defeat',
                       'defeated', 'defeating', 'defeats', 'defect', 'defective', 'defects', 'defend', 'defendant',
                       'defendants', 'defended', 'defending', 'defends', 'defensive', 'defer', 'deficiencies',
                       'deficiency', 'deficient', 'deficit', 'deficits', 'defraud', 'defrauded', 'defrauding',
                       'defrauds', 'defunct', 'degradation', 'degradations', 'degrade', 'degraded', 'degrades',
                       'degrading', 'delay', 'delayed', 'delaying', 'delays', 'deleterious', 'deliberate',
                       'deliberated', 'deliberately', 'delinquencies', 'delinquency', 'delinquent', 'delinquently',
                       'delinquents', 'delist', 'delisted', 'delisting', 'delists', 'demise', 'demised', 'demises',
                       'demising', 'demolish', 'demolished', 'demolishes', 'demolishing', 'demolition', 'demolitions',
                       'demote', 'demoted', 'demotes', 'demoting', 'demotion', 'demotions', 'denial', 'denials',
                       'denied', 'denies', 'denigrate', 'denigrated', 'denigrates', 'denigrating', 'denigration',
                       'deny', 'denying', 'deplete', 'depleted', 'depletes', 'depleting', 'depletion', 'depletions',
                       'deprecation', 'depress', 'depressed', 'depresses', 'depressing', 'deprivation', 'deprive',
                       'deprived', 'deprives', 'depriving', 'derelict', 'dereliction', 'derogatory', 'destabilization',
                       'destabilize', 'destabilized', 'destabilizing', 'destroy', 'destroyed', 'destroying', 'destroys',
                       'destruction', 'destructive', 'detain', 'detained', 'detention', 'detentions', 'deter',
                       'deteriorate', 'deteriorated', 'deteriorates', 'deteriorating', 'deterioration',
                       'deteriorations', 'deterred', 'deterrence', 'deterrences', 'deterrent', 'deterrents',
                       'deterring', 'deters', 'detract', 'detracted', 'detracting', 'detriment', 'detrimental',
                       'detrimentally', 'detriments', 'devalue', 'devalued', 'devalues', 'devaluing', 'devastate',
                       'devastated', 'devastating', 'devastation', 'deviate', 'deviated', 'deviates', 'deviating',
                       'deviation', 'deviations', 'devolve', 'devolved', 'devolves', 'devolving', 'difficult',
                       'difficulties', 'difficultly', 'difficulty', 'diminish', 'diminished', 'diminishes',
                       'diminishing', 'diminution', 'disadvantage', 'disadvantaged', 'disadvantageous', 'disadvantages',
                       'disaffiliation', 'disagree', 'disagreeable', 'disagreed', 'disagreeing', 'disagreement',
                       'disagreements', 'disagrees', 'disallow', 'disallowance', 'disallowances', 'disallowed',
                       'disallowing', 'disallows', 'disappear', 'disappearance', 'disappearances', 'disappeared',
                       'disappearing', 'disappears', 'disappoint', 'disappointed', 'disappointing', 'disappointingly',
                       'disappointment', 'disappointments', 'disappoints', 'disapproval', 'disapprovals', 'disapprove',
                       'disapproved', 'disapproves', 'disapproving', 'disassociates', 'disassociating',
                       'disassociation', 'disassociations', 'disaster', 'disasters', 'disastrous', 'disastrously',
                       'disavow', 'disavowal', 'disavowed', 'disavowing', 'disavows', 'disciplinary', 'disclaim',
                       'disclaimed', 'disclaimer', 'disclaimers', 'disclaiming', 'disclaims', 'disclose', 'disclosed',
                       'discloses', 'disclosing', 'discontinuance', 'discontinuances', 'discontinuation',
                       'discontinuations', 'discontinue', 'discontinued', 'discontinues', 'discontinuing', 'discourage',
                       'discouraged', 'discourages', 'discouraging', 'discredit', 'discredited', 'discrediting',
                       'discredits', 'discrepancies', 'discrepancy', 'disfavor', 'disfavored', 'disfavoring',
                       'disfavors', 'disgorge', 'disgorged', 'disgorgement', 'disgorgements', 'disgorges', 'disgorging',
                       'disgrace', 'disgraceful', 'disgracefully', 'dishonest', 'dishonestly', 'dishonesty', 'dishonor',
                       'dishonorable', 'dishonorably', 'dishonored', 'dishonoring', 'dishonors', 'disincentives',
                       'disinterested', 'disinterestedly', 'disinterestedness', 'disloyal', 'disloyally', 'disloyalty',
                       'dismal', 'dismally', 'dismiss', 'dismissal', 'dismissals', 'dismissed', 'dismisses',
                       'dismissing', 'disorderly', 'disparage', 'disparaged', 'disparagement', 'disparagements',
                       'disparages', 'disparaging', 'disparagingly', 'disparities', 'disparity', 'displace',
                       'displaced', 'displacement', 'displacements', 'displaces', 'displacing', 'dispose', 'dispossess',
                       'dispossessed', 'dispossesses', 'dispossessing', 'disproportion', 'disproportional',
                       'disproportionate', 'disproportionately', 'dispute', 'disputed', 'disputes', 'disputing',
                       'disqualification', 'disqualifications', 'disqualified', 'disqualifies', 'disqualify',
                       'disqualifying', 'disregard', 'disregarded', 'disregarding', 'disregards', 'disreputable',
                       'disrepute', 'disrupt', 'disrupted', 'disrupting', 'disruption', 'disruptions', 'disruptive',
                       'disrupts', 'dissatisfaction', 'dissatisfied', 'dissent', 'dissented', 'dissenter', 'dissenters',
                       'dissenting', 'dissents', 'dissident', 'dissidents', 'dissolution', 'dissolutions', 'distort',
                       'distorted', 'distorting', 'distortion', 'distortions', 'distorts', 'distract', 'distracted',
                       'distracting', 'distraction', 'distractions', 'distracts', 'distress', 'distressed', 'disturb',
                       'disturbance', 'disturbances', 'disturbed', 'disturbing', 'disturbs', 'diversion', 'divert',
                       'diverted', 'diverting', 'diverts', 'divest', 'divested', 'divesting', 'divestiture',
                       'divestitures', 'divestment', 'divestments', 'divests', 'divorce', 'divorced', 'divulge',
                       'divulged', 'divulges', 'divulging', 'doubt', 'doubted', 'doubtful', 'doubts', 'downgrade',
                       'downgraded', 'downgrades', 'downgrading', 'downsize', 'downsized', 'downsizes', 'downsizing',
                       'downsizings', 'downtime', 'downtimes', 'downturn', 'downturns', 'downward', 'downwards', 'drag',
                       'drastic', 'drastically', 'drawback', 'drawbacks', 'dropped', 'drought', 'droughts', 'duress',
                       'dysfunction', 'dysfunctional', 'dysfunctions', 'easing', 'egregious', 'egregiously', 'embargo',
                       'embargoed', 'embargoes', 'embargoing', 'embarrass', 'embarrassed', 'embarrasses',
                       'embarrassing', 'embarrassment', 'embarrassments', 'embezzle', 'embezzled', 'embezzlement',
                       'embezzlements', 'embezzler', 'embezzles', 'embezzling', 'encroach', 'encroached', 'encroaches',
                       'encroaching', 'encroachment', 'encroachments', 'encumber', 'encumbered', 'encumbering',
                       'encumbers', 'encumbrance', 'encumbrances', 'endanger', 'endangered', 'endangering',
                       'endangerment', 'endangers', 'enjoin', 'enjoined', 'enjoining', 'enjoins', 'erode', 'eroded',
                       'erodes', 'eroding', 'erosion', 'erratic', 'erratically', 'erred', 'erring', 'erroneous',
                       'erroneously', 'error', 'errors', 'errs', 'escalate', 'escalated', 'escalates', 'escalating',
                       'evade', 'evaded', 'evades', 'evading', 'evasion', 'evasions', 'evasive', 'evict', 'evicted',
                       'evicting', 'eviction', 'evictions', 'evicts', 'exacerbate', 'exacerbated', 'exacerbates',
                       'exacerbating', 'exacerbation', 'exacerbations', 'exaggerate', 'exaggerated', 'exaggerates',
                       'exaggerating', 'exaggeration', 'excessive', 'excessively', 'exculpate', 'exculpated',
                       'exculpates', 'exculpating', 'exculpation', 'exculpations', 'exculpatory', 'exonerate',
                       'exonerated', 'exonerates', 'exonerating', 'exoneration', 'exonerations', 'exploit',
                       'exploitation', 'exploitations', 'exploitative', 'exploited', 'exploiting', 'exploits', 'expose',
                       'exposed', 'exposes', 'exposing', 'expropriate', 'expropriated', 'expropriates', 'expropriating',
                       'expropriation', 'expropriations', 'expulsion', 'expulsions', 'extenuating', 'fail', 'failed',
                       'failing', 'failings', 'fails', 'failure', 'failures', 'fallout', 'false', 'falsely',
                       'falsification', 'falsifications', 'falsified', 'falsifies', 'falsify', 'falsifying', 'falsity',
                       'fatalities', 'fatality', 'fatally', 'fault', 'faulted', 'faults', 'faulty', 'fear', 'fears',
                       'felonies', 'felonious', 'felony', 'fictitious', 'fined', 'fines', 'fired', 'firing', 'flaw',
                       'flawed', 'flaws', 'forbid', 'forbidden', 'forbidding', 'forbids', 'force', 'forced', 'forcing',
                       'foreclose', 'foreclosed', 'forecloses', 'foreclosing', 'foreclosure', 'foreclosures', 'forego',
                       'foregoes', 'foregone', 'forestall', 'forestalled', 'forestalling', 'forestalls', 'forfeit',
                       'forfeited', 'forfeiting', 'forfeits', 'forfeiture', 'forfeitures', 'forgers', 'forgery',
                       'fraud', 'frauds', 'fraudulence', 'fraudulent', 'fraudulently', 'frivolous', 'frivolously',
                       'frustrate', 'frustrated', 'frustrates', 'frustrating', 'frustratingly', 'frustration',
                       'frustrations', 'fugitive', 'fugitives', 'gratuitous', 'gratuitously', 'grievance', 'grievances',
                       'grossly', 'groundless', 'guilty', 'halt', 'halted', 'hamper', 'hampered', 'hampering',
                       'hampers', 'harass', 'harassed', 'harassing', 'harassment', 'hardship', 'hardships', 'harm',
                       'harmed', 'harmful', 'harmfully', 'harming', 'harms', 'harsh', 'harsher', 'harshest', 'harshly',
                       'harshness', 'hazard', 'hazardous', 'hazards', 'hinder', 'hindered', 'hindering', 'hinders',
                       'hindrance', 'hindrances', 'hostile', 'hostility', 'hurt', 'hurting', 'idle', 'idled', 'idling',
                       'ignore', 'ignored', 'ignores', 'ignoring', 'ill', 'illegal', 'illegalities', 'illegality',
                       'illegally', 'illegible', 'illicit', 'illicitly', 'illiquid', 'illiquidity', 'imbalance',
                       'imbalances', 'immature', 'immoral', 'impair', 'impaired', 'impairing', 'impairment',
                       'impairments', 'impairs', 'impasse', 'impasses', 'impede', 'impeded', 'impedes', 'impediment',
                       'impediments', 'impeding', 'impending', 'imperative', 'imperfection', 'imperfections', 'imperil',
                       'impermissible', 'implicate', 'implicated', 'implicates', 'implicating', 'impossibility',
                       'impossible', 'impound', 'impounded', 'impounding', 'impounds', 'impracticable', 'impractical',
                       'impracticalities', 'impracticality', 'imprisonment', 'improper', 'improperly', 'improprieties',
                       'impropriety', 'imprudent', 'imprudently', 'inability', 'inaccessible', 'inaccuracies',
                       'inaccuracy', 'inaccurate', 'inaccurately', 'inaction', 'inactions', 'inactivate', 'inactivated',
                       'inactivates', 'inactivating', 'inactivation', 'inactivations', 'inactivity', 'inadequacies',
                       'inadequacy', 'inadequate', 'inadequately', 'inadvertent', 'inadvertently', 'inadvisability',
                       'inadvisable', 'inappropriate', 'inappropriately', 'inattention', 'incapable', 'incapacitated',
                       'incapacity', 'incarcerate', 'incarcerated', 'incarcerates', 'incarcerating', 'incarceration',
                       'incarcerations', 'incidence', 'incidences', 'incident', 'incidents', 'incompatibilities',
                       'incompatibility', 'incompatible', 'incompetence', 'incompetency', 'incompetent',
                       'incompetently', 'incompetents', 'incomplete', 'incompletely', 'incompleteness', 'inconclusive',
                       'inconsistencies', 'inconsistency', 'inconsistent', 'inconsistently', 'inconvenience',
                       'inconveniences', 'inconvenient', 'incorrect', 'incorrectly', 'incorrectness', 'indecency',
                       'indecent', 'indefeasible', 'indefeasibly', 'indict', 'indictable', 'indicted', 'indicting',
                       'indictment', 'indictments', 'ineffective', 'ineffectively', 'ineffectiveness', 'inefficiencies',
                       'inefficiency', 'inefficient', 'inefficiently', 'ineligibility', 'ineligible', 'inequitable',
                       'inequitably', 'inequities', 'inequity', 'inevitable', 'inexperience', 'inexperienced',
                       'inferior', 'inflicted', 'infraction', 'infractions', 'infringe', 'infringed', 'infringement',
                       'infringements', 'infringes', 'infringing', 'inhibited', 'inimical', 'injunction', 'injunctions',
                       'injure', 'injured', 'injures', 'injuries', 'injuring', 'injurious', 'injury', 'inordinate',
                       'inordinately', 'inquiry', 'insecure', 'insensitive', 'insolvencies', 'insolvency', 'insolvent',
                       'instability', 'insubordination', 'insufficiency', 'insufficient', 'insufficiently',
                       'insurrection', 'insurrections', 'intentional', 'interfere', 'interfered', 'interference',
                       'interferences', 'interferes', 'interfering', 'intermittent', 'intermittently', 'interrupt',
                       'interrupted', 'interrupting', 'interruption', 'interruptions', 'interrupts', 'intimidation',
                       'intrusion', 'invalid', 'invalidate', 'invalidated', 'invalidates', 'invalidating',
                       'invalidation', 'invalidity', 'investigate', 'investigated', 'investigates', 'investigating',
                       'investigation', 'investigations', 'involuntarily', 'involuntary', 'irreconcilable',
                       'irreconcilably', 'irrecoverable', 'irrecoverably', 'irregular', 'irregularities',
                       'irregularity', 'irregularly', 'irreparable', 'irreparably', 'irreversible', 'jeopardize',
                       'jeopardized', 'justifiable', 'kickback', 'kickbacks', 'knowingly', 'lack', 'lacked', 'lacking',
                       'lackluster', 'lacks', 'lag', 'lagged', 'lagging', 'lags', 'lapse', 'lapsed', 'lapses',
                       'lapsing', 'late', 'laundering', 'layoff', 'layoffs', 'lie', 'limitation', 'limitations',
                       'lingering', 'liquidate', 'liquidated', 'liquidates', 'liquidating', 'liquidation',
                       'liquidations', 'liquidator', 'liquidators', 'litigant', 'litigants', 'litigate', 'litigated',
                       'litigates', 'litigating', 'litigation', 'litigations', 'lockout', 'lockouts', 'lose', 'loses',
                       'losing', 'loss', 'losses', 'lost', 'lying', 'malfeasance', 'malfunction', 'malfunctioned',
                       'malfunctioning', 'malfunctions', 'malice', 'malicious', 'maliciously', 'malpractice',
                       'manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', 'manipulations',
                       'manipulative', 'markdown', 'markdowns', 'misapplication', 'misapplications', 'misapplied',
                       'misapplies', 'misapply', 'misapplying', 'misappropriate', 'misappropriated', 'misappropriates',
                       'misappropriating', 'misappropriation', 'misappropriations', 'misbranded', 'miscalculate',
                       'miscalculated', 'miscalculates', 'miscalculating', 'miscalculation', 'miscalculations',
                       'mischaracterization', 'mischief', 'misclassification', 'misclassifications', 'misclassified',
                       'misclassify', 'miscommunication', 'misconduct', 'misdated', 'misdemeanor', 'misdemeanors',
                       'misdirected', 'mishandle', 'mishandled', 'mishandles', 'mishandling', 'misinform',
                       'misinformation', 'misinformed', 'misinforming', 'misinforms', 'misinterpret',
                       'misinterpretation', 'misinterpretations', 'misinterpreted', 'misinterpreting', 'misinterprets',
                       'misjudge', 'misjudged', 'misjudges', 'misjudging', 'misjudgment', 'misjudgments', 'mislabel',
                       'mislabeled', 'mislabeling', 'mislabelled', 'mislabels', 'mislead', 'misleading', 'misleadingly',
                       'misleads', 'misled', 'mismanage', 'mismanaged', 'mismanagement', 'mismanages', 'mismanaging',
                       'mismatch', 'mismatched', 'mismatches', 'mismatching', 'misplaced', 'misprice', 'mispricing',
                       'mispricings', 'misrepresent', 'misrepresentation', 'misrepresentations', 'misrepresented',
                       'misrepresenting', 'misrepresents', 'miss', 'missed', 'misses', 'misstate', 'misstated',
                       'misstatement', 'misstatements', 'misstates', 'misstating', 'misstep', 'missteps', 'mistake',
                       'mistaken', 'mistakenly', 'mistakes', 'mistaking', 'mistrial', 'mistrials', 'misunderstand',
                       'misunderstanding', 'misunderstandings', 'misunderstood', 'misuse', 'misused', 'misuses',
                       'misusing', 'monopolistic', 'monopolists', 'monopolization', 'monopolize', 'monopolized',
                       'monopolizes', 'monopolizing', 'monopoly', 'moratoria', 'moratorium', 'moratoriums',
                       'mothballed', 'mothballing', 'negative', 'negatively', 'negatives', 'neglect', 'neglected',
                       'neglectful', 'neglecting', 'neglects', 'negligence', 'negligences', 'negligent', 'negligently',
                       'nonattainment', 'noncompetitive', 'noncompliance', 'noncompliances', 'noncompliant',
                       'noncomplying', 'nonconforming', 'nonconformities', 'nonconformity', 'nondisclosure',
                       'nonfunctional', 'nonpayment', 'nonpayments', 'nonperformance', 'nonperformances',
                       'nonperforming', 'nonproducing', 'nonproductive', 'nonrecoverable', 'nonrenewal', 'nuisance',
                       'nuisances', 'nullification', 'nullifications', 'nullified', 'nullifies', 'nullify',
                       'nullifying', 'objected', 'objecting', 'objection', 'objectionable', 'objectionably',
                       'objections', 'obscene', 'obscenity', 'obsolescence', 'obsolete', 'obstacle', 'obstacles',
                       'obstruct', 'obstructed', 'obstructing', 'obstruction', 'obstructions', 'offence', 'offences',
                       'offend', 'offended', 'offender', 'offenders', 'offending', 'offends', 'omission', 'omissions',
                       'omit', 'omits', 'omitted', 'omitting', 'onerous', 'opportunistic', 'opportunistically',
                       'oppose', 'opposed', 'opposes', 'opposing', 'opposition', 'oppositions', 'outage', 'outages',
                       'outdated', 'outmoded', 'overage', 'overages', 'overbuild', 'overbuilding', 'overbuilds',
                       'overbuilt', 'overburden', 'overburdened', 'overburdening', 'overcapacities', 'overcapacity',
                       'overcharge', 'overcharged', 'overcharges', 'overcharging', 'overcome', 'overcomes',
                       'overcoming', 'overdue', 'overestimate', 'overestimated', 'overestimates', 'overestimating',
                       'overestimation', 'overestimations', 'overload', 'overloaded', 'overloading', 'overloads',
                       'overlook', 'overlooked', 'overlooking', 'overlooks', 'overpaid', 'overpayment', 'overpayments',
                       'overproduced', 'overproduces', 'overproducing', 'overproduction', 'overrun', 'overrunning',
                       'overruns', 'overshadow', 'overshadowed', 'overshadowing', 'overshadows', 'overstate',
                       'overstated', 'overstatement', 'overstatements', 'overstates', 'overstating', 'oversupplied',
                       'oversupplies', 'oversupply', 'oversupplying', 'overtly', 'overturn', 'overturned',
                       'overturning', 'overturns', 'overvalue', 'overvalued', 'overvaluing', 'panic', 'panics',
                       'penalize', 'penalized', 'penalizes', 'penalizing', 'penalties', 'penalty', 'peril', 'perils',
                       'perjury', 'perpetrate', 'perpetrated', 'perpetrates', 'perpetrating', 'perpetration', 'persist',
                       'persisted', 'persistence', 'persistent', 'persistently', 'persisting', 'persists', 'pervasive',
                       'pervasively', 'pervasiveness', 'petty', 'picket', 'picketed', 'picketing', 'plaintiff',
                       'plaintiffs', 'plea', 'plead', 'pleaded', 'pleading', 'pleadings', 'pleads', 'pleas', 'pled',
                       'poor', 'poorly', 'poses', 'posing', 'postpone', 'postponed', 'postponement', 'postponements',
                       'postpones', 'postponing', 'precipitated', 'precipitous', 'precipitously', 'preclude',
                       'precluded', 'precludes', 'precluding', 'predatory', 'prejudice', 'prejudiced', 'prejudices',
                       'prejudicial', 'prejudicing', 'premature', 'prematurely', 'pressing', 'pretrial', 'preventing',
                       'prevention', 'prevents', 'problem', 'problematic', 'problematical', 'problems', 'prolong',
                       'prolongation', 'prolongations', 'prolonged', 'prolonging', 'prolongs', 'prone', 'prosecute',
                       'prosecuted', 'prosecutes', 'prosecuting', 'prosecution', 'prosecutions', 'protest', 'protested',
                       'protester', 'protesters', 'protesting', 'protestor', 'protestors', 'protests', 'protracted',
                       'protraction', 'provoke', 'provoked', 'provokes', 'provoking', 'punished', 'punishes',
                       'punishing', 'punishment', 'punishments', 'punitive', 'purport', 'purported', 'purportedly',
                       'purporting', 'purports', 'question', 'questionable', 'questionably', 'questioned',
                       'questioning', 'questions', 'quit', 'quitting', 'racketeer', 'racketeering', 'rationalization',
                       'rationalizations', 'rationalize', 'rationalized', 'rationalizes', 'rationalizing',
                       'reassessment', 'reassessments', 'reassign', 'reassigned', 'reassigning', 'reassignment',
                       'reassignments', 'reassigns', 'recall', 'recalled', 'recalling', 'recalls', 'recession',
                       'recessionary', 'recessions', 'reckless', 'recklessly', 'recklessness', 'redact', 'redacted',
                       'redacting', 'redaction', 'redactions', 'redefault', 'redefaulted', 'redefaults', 'redress',
                       'redressed', 'redresses', 'redressing', 'refusal', 'refusals', 'refuse', 'refused', 'refuses',
                       'refusing', 'reject', 'rejected', 'rejecting', 'rejection', 'rejections', 'rejects',
                       'relinquish', 'relinquished', 'relinquishes', 'relinquishing', 'relinquishment',
                       'relinquishments', 'reluctance', 'reluctant', 'renegotiate', 'renegotiated', 'renegotiates',
                       'renegotiating', 'renegotiation', 'renegotiations', 'renounce', 'renounced', 'renouncement',
                       'renouncements', 'renounces', 'renouncing', 'reparation', 'reparations', 'repossessed',
                       'repossesses', 'repossessing', 'repossession', 'repossessions', 'repudiate', 'repudiated',
                       'repudiates', 'repudiating', 'repudiation', 'repudiations', 'resign', 'resignation',
                       'resignations', 'resigned', 'resigning', 'resigns', 'restate', 'restated', 'restatement',
                       'restatements', 'restates', 'restating', 'restructure', 'restructured', 'restructures',
                       'restructuring', 'restructurings', 'retaliate', 'retaliated', 'retaliates', 'retaliating',
                       'retaliation', 'retaliations', 'retaliatory', 'retribution', 'retributions', 'revocation',
                       'revocations', 'revoke', 'revoked', 'revokes', 'revoking', 'ridicule', 'ridiculed', 'ridicules',
                       'ridiculing', 'riskier', 'riskiest', 'risky', 'sabotage', 'sacrifice', 'sacrificed',
                       'sacrifices', 'sacrificial', 'sacrificing', 'scandalous', 'scandals', 'scrutinize',
                       'scrutinized', 'scrutinizes', 'scrutinizing', 'scrutiny', 'secrecy', 'seize', 'seized', 'seizes',
                       'seizing', 'sentenced', 'sentencing', 'serious', 'seriously', 'seriousness', 'setback',
                       'setbacks', 'sever', 'severe', 'severed', 'severely', 'severities', 'severity', 'sharply',
                       'shocked', 'shortage', 'shortages', 'shortfall', 'shortfalls', 'shrinkage', 'shrinkages', 'shut',
                       'shutdown', 'shutdowns', 'shuts', 'shutting', 'slander', 'slandered', 'slanderous', 'slanders',
                       'slippage', 'slippages', 'slow', 'slowdown', 'slowdowns', 'slowed', 'slower', 'slowest',
                       'slowing', 'slowly', 'slowness', 'sluggish', 'sluggishly', 'sluggishness', 'solvencies',
                       'solvency', 'spam', 'spammers', 'spamming', 'staggering', 'stagnant', 'stagnate', 'stagnated',
                       'stagnates', 'stagnating', 'stagnation', 'standstill', 'standstills', 'stolen', 'stoppage',
                       'stoppages', 'stopped', 'stopping', 'stops', 'strain', 'strained', 'straining', 'strains',
                       'stress', 'stressed', 'stresses', 'stressful', 'stressing', 'stringent', 'subjected',
                       'subjecting', 'subjection', 'subpoena', 'subpoenaed', 'subpoenas', 'substandard', 'sue', 'sued',
                       'sues', 'suffer', 'suffered', 'suffering', 'suffers', 'suing', 'summoned', 'summoning',
                       'summons', 'summonses', 'susceptibility', 'susceptible', 'suspect', 'suspected', 'suspects',
                       'suspend', 'suspended', 'suspending', 'suspends', 'suspension', 'suspensions', 'suspicion',
                       'suspicions', 'suspicious', 'suspiciously', 'taint', 'tainted', 'tainting', 'taints', 'tampered',
                       'tense', 'terminate', 'terminated', 'terminates', 'terminating', 'termination', 'terminations',
                       'testify', 'testifying', 'threat', 'threaten', 'threatened', 'threatening', 'threatens',
                       'threats', 'tightening', 'tolerate', 'tolerated', 'tolerates', 'tolerating', 'toleration',
                       'tortuous', 'tortuously', 'tragedies', 'tragedy', 'tragic', 'tragically', 'traumatic', 'trouble',
                       'troubled', 'troubles', 'turbulence', 'turmoil', 'unable', 'unacceptable', 'unacceptably',
                       'unaccounted', 'unannounced', 'unanticipated', 'unapproved', 'unattractive', 'unauthorized',
                       'unavailability', 'unavailable', 'unavoidable', 'unavoidably', 'unaware', 'uncollectable',
                       'uncollected', 'uncollectibility', 'uncollectible', 'uncollectibles', 'uncompetitive',
                       'uncompleted', 'unconscionable', 'unconscionably', 'uncontrollable', 'uncontrollably',
                       'uncontrolled', 'uncorrected', 'uncover', 'uncovered', 'uncovering', 'uncovers', 'undeliverable',
                       'undelivered', 'undercapitalized', 'undercut', 'undercuts', 'undercutting', 'underestimate',
                       'underestimated', 'underestimates', 'underestimating', 'underestimation', 'underfunded',
                       'underinsured', 'undermine', 'undermined', 'undermines', 'undermining', 'underpaid',
                       'underpayment', 'underpayments', 'underpays', 'underperform', 'underperformance',
                       'underperformed', 'underperforming', 'underperforms', 'underproduced', 'underproduction',
                       'underreporting', 'understate', 'understated', 'understatement', 'understatements',
                       'understates', 'understating', 'underutilization', 'underutilized', 'undesirable', 'undesired',
                       'undetected', 'undetermined', 'undisclosed', 'undocumented', 'undue', 'unduly', 'uneconomic',
                       'uneconomical', 'uneconomically', 'unemployed', 'unemployment', 'unethical', 'unethically',
                       'unexcused', 'unexpected', 'unexpectedly', 'unfair', 'unfairly', 'unfavorability', 'unfavorable',
                       'unfavorably', 'unfavourable', 'unfeasible', 'unfit', 'unfitness', 'unforeseeable', 'unforeseen',
                       'unforseen', 'unfortunate', 'unfortunately', 'unfounded', 'unfriendly', 'unfulfilled',
                       'unfunded', 'uninsured', 'unintended', 'unintentional', 'unintentionally', 'unjust',
                       'unjustifiable', 'unjustifiably', 'unjustified', 'unjustly', 'unknowing', 'unknowingly',
                       'unlawful', 'unlawfully', 'unlicensed', 'unliquidated', 'unmarketable', 'unmerchantable',
                       'unmeritorious', 'unnecessarily', 'unnecessary', 'unneeded', 'unobtainable', 'unoccupied',
                       'unpaid', 'unperformed', 'unplanned', 'unpopular', 'unpredictability', 'unpredictable',
                       'unpredictably', 'unpredicted', 'unproductive', 'unprofitability', 'unprofitable', 'unqualified',
                       'unrealistic', 'unreasonable', 'unreasonableness', 'unreasonably', 'unreceptive',
                       'unrecoverable', 'unrecovered', 'unreimbursed', 'unreliable', 'unremedied', 'unreported',
                       'unresolved', 'unrest', 'unsafe', 'unsalable', 'unsaleable', 'unsatisfactory', 'unsatisfied',
                       'unsavory', 'unscheduled', 'unsellable', 'unsold', 'unsound', 'unstabilized', 'unstable',
                       'unsubstantiated', 'unsuccessful', 'unsuccessfully', 'unsuitability', 'unsuitable', 'unsuitably',
                       'unsuited', 'unsure', 'unsuspected', 'unsuspecting', 'unsustainable', 'untenable', 'untimely',
                       'untrusted', 'untruth', 'untruthful', 'untruthfully', 'untruthfulness', 'untruths', 'unusable',
                       'unwanted', 'unwarranted', 'unwelcome', 'unwilling', 'unwillingness', 'upset', 'urgency',
                       'urgent', 'usurious', 'usurp', 'usurped', 'usurping', 'usurps', 'usury', 'vandalism', 'verdict',
                       'verdicts', 'vetoed', 'victims', 'violate', 'violated', 'violates', 'violating', 'violation',
                       'violations', 'violative', 'violator', 'violators', 'violence', 'violent', 'violently',
                       'vitiate', 'vitiated', 'vitiates', 'vitiating', 'vitiation', 'voided', 'voiding', 'volatile',
                       'volatility', 'vulnerabilities', 'vulnerability', 'vulnerable', 'vulnerably', 'warn', 'warned',
                       'warning', 'warnings', 'warns', 'wasted', 'wasteful', 'wasting', 'weak', 'weaken', 'weakened',
                       'weakening', 'weakens', 'weaker', 'weakest', 'weakly', 'weakness', 'weaknesses', 'willfully',
                       'worries', 'worry', 'worrying', 'worse', 'worsen', 'worsened', 'worsening', 'worsens', 'worst',
                       'worthless', 'writedown', 'writedowns', 'writeoff', 'writeoffs', 'wrong', 'wrongdoing',
                       'wrongdoings', 'wrongful', 'wrongfully', 'wrongly', 'negative', 'negatives', 'fail', 'fails', 'failing', 'failure', 'weak', 'weakness', 'weaknesses',
                      'difficult', 'difficulty', 'hurdle', 'hurdles', 'obstacle', 'obstacles', 'slump', 'slumps',
                      'slumping', 'slumped', 'uncertain', 'uncertainty', 'unsettled', 'unfavorable', 'downturn',
                      'depressed', 'disappoint', 'disappoints', 'disappointing', 'disappointed', 'disappointment',
                      'risk', 'risks', 'risky', 'threat', 'threats', 'penalty', 'penalties', 'down', 'decrease',
                      'decreases', 'decreasing', 'decreased', 'decline', 'declines', 'declining', 'declined', 'fall',
                      'falls', 'falling', 'fell', 'fallen', 'drop', 'drops', 'dropping', 'dropped', 'deteriorate',
                      'deteriorates', 'deteriorating', 'deteriorated', 'worsen', 'worsens', 'worsening', 'weaken',
                      'weakens', 'weakening', 'weakened', 'worse', 'worst', 'low', 'lower', 'lowest', 'less', 'least',
                      'smaller', 'smallest', 'shrink', 'shrinks', 'shrinking', 'shrunk', 'below', 'under', 'challenge',
                      'challenges', 'challenging', 'challenged'] 
          

In [110]:
# Introduce negative financial keywords
Positive = ['able', 'abundance', 'abundant', 'acclaimed', 'accomplish', 'accomplished', 'accomplishes',
                       'accomplishing', 'accomplishment', 'accomplishments', 'achieve', 'achieved', 'achievement',
                       'achievements', 'achieves', 'achieving', 'adequately', 'advancement', 'advancements', 'advances',
                       'advancing', 'advantage', 'advantaged', 'advantageous', 'advantageously', 'advantages',
                       'alliance', 'alliances', 'assure', 'assured', 'assures', 'assuring', 'attain', 'attained',
                       'attaining', 'attainment', 'attainments', 'attains', 'attractive', 'attractiveness', 'beautiful',
                       'beautifully', 'beneficial', 'beneficially', 'benefit', 'benefited', 'benefiting', 'benefitted',
                       'benefitting', 'best', 'better', 'bolstered', 'bolstering', 'bolsters', 'boom', 'booming',
                       'boost', 'boosted', 'breakthrough', 'breakthroughs', 'brilliant', 'charitable', 'collaborate',
                       'collaborated', 'collaborates', 'collaborating', 'collaboration', 'collaborations',
                       'collaborative', 'collaborator', 'collaborators', 'compliment', 'complimentary', 'complimented',
                       'complimenting', 'compliments', 'conclusive', 'conclusively', 'conducive', 'confident',
                       'constructive', 'constructively', 'courteous', 'creative', 'creatively', 'creativeness',
                       'creativity', 'delight', 'delighted', 'delightful', 'delightfully', 'delighting', 'delights',
                       'dependability', 'dependable', 'desirable', 'desired', 'despite', 'destined', 'diligent',
                       'diligently', 'distinction', 'distinctions', 'distinctive', 'distinctively', 'distinctiveness',
                       'dream', 'easier', 'easily', 'easy', 'effective', 'efficiencies', 'efficiency', 'efficient',
                       'efficiently', 'empower', 'empowered', 'empowering', 'empowers', 'enable', 'enabled', 'enables',
                       'enabling', 'encouraged', 'encouragement', 'encourages', 'encouraging', 'enhance', 'enhanced',
                       'enhancement', 'enhancements', 'enhances', 'enhancing', 'enjoy', 'enjoyable', 'enjoyably',
                       'enjoyed', 'enjoying', 'enjoyment', 'enjoys', 'enthusiasm', 'enthusiastic', 'enthusiastically',
                       'excellence', 'excellent', 'excelling', 'excels', 'exceptional', 'exceptionally', 'excited',
                       'excitement', 'exciting', 'exclusive', 'exclusively', 'exclusiveness', 'exclusives',
                       'exclusivity', 'exemplary', 'fantastic', 'favorable', 'favorably', 'favored', 'favoring',
                       'favorite', 'favorites', 'friendly', 'gain', 'gained', 'gaining', 'gains', 'good', 'great',
                       'greater', 'greatest', 'greatly', 'greatness', 'happiest', 'happily', 'happiness', 'happy',
                       'highest', 'honor', 'honorable', 'honored', 'honoring', 'honors', 'ideal', 'impress',
                       'impressed', 'impresses', 'impressing', 'impressive', 'impressively', 'improve', 'improved',
                       'improvement', 'improvements', 'improves', 'improving', 'incredible', 'incredibly',
                       'influential', 'informative', 'ingenuity', 'innovate', 'innovated', 'innovates', 'innovating',
                       'innovation', 'innovations', 'innovative', 'innovativeness', 'innovator', 'innovators',
                       'insightful', 'inspiration', 'inspirational', 'integrity', 'invent', 'invented', 'inventing',
                       'invention', 'inventions', 'inventive', 'inventiveness', 'inventor', 'inventors', 'leadership',
                       'leading', 'loyal', 'lucrative', 'meritorious', 'opportunities', 'opportunity', 'optimistic',
                       'outperform', 'outperformed', 'outperforming', 'outperforms', 'perfect', 'perfected',
                       'perfectly', 'perfects', 'pleasant', 'pleasantly', 'pleased', 'pleasure', 'plentiful', 'popular',
                       'popularity', 'positive', 'positively', 'preeminence', 'preeminent', 'premier', 'premiere',
                       'prestige', 'prestigious', 'proactive', 'proactively', 'proficiency', 'proficient',
                       'proficiently', 'profitability', 'profitable', 'profitably', 'progress', 'progressed',
                       'progresses', 'progressing', 'prospered', 'prospering', 'prosperity', 'prosperous', 'prospers',
                       'rebound', 'rebounded', 'rebounding', 'receptive', 'regain', 'regained', 'regaining', 'resolve',
                       'revolutionize', 'revolutionized', 'revolutionizes', 'revolutionizing', 'reward', 'rewarded',
                       'rewarding', 'rewards', 'satisfaction', 'satisfactorily', 'satisfactory', 'satisfied',
                       'satisfies', 'satisfy', 'satisfying', 'smooth', 'smoothing', 'smoothly', 'smooths', 'solves',
                       'solving', 'spectacular', 'spectacularly', 'stability', 'stabilization', 'stabilizations',
                       'stabilize', 'stabilized', 'stabilizes', 'stabilizing', 'stable', 'strength', 'strengthen',
                       'strengthened', 'strengthening', 'strengthens', 'strengths', 'strong', 'stronger', 'strongest',
                       'succeed', 'succeeded', 'succeeding', 'succeeds', 'success', 'successes', 'successful',
                       'successfully', 'superior', 'surpass', 'surpassed', 'surpasses', 'surpassing', 'transparency',
                       'tremendous', 'tremendously', 'unmatched', 'unparalleled', 'unsurpassed', 'upturn', 'upturns',
                       'valuable', 'versatile', 'versatility', 'vibrancy', 'vibrant', 'win', 'winner', 'winners',
                       'winning', 'worthy', 'positive', 'positives', 'success', 'successes', 'successful', 'succeed', 'succeeds',
                      'succeeding', 'succeeded', 'accomplish', 'accomplishes', 'accomplishing', 'accomplished',
                      'accomplishment', 'accomplishments', 'strong', 'strength', 'strengths', 'certain', 'certainty',
                      'definite', 'solid', 'excellent', 'good', 'leading', 'achieve', 'achieves', 'achieved',
                      'achieving', 'achievement', 'achievements', 'progress', 'progressing', 'deliver', 'delivers',
                      'delivered', 'delivering', 'leader', 'leading', 'pleased', 'reward', 'rewards', 'rewarding',
                      'rewarded', 'opportunity', 'opportunities', 'enjoy', 'enjoys', 'enjoying', 'enjoyed',
                      'encouraged', 'encouraging', 'up', 'increase', 'increases', 'increasing', 'increased', 'rise',
                      'rises', 'rising', 'rose', 'risen', 'improve', 'improves', 'improving', 'improved', 'improvement',
                      'improvements', 'strengthen', 'strengthens', 'strengthening', 'strengthened', 'stronger',
                      'strongest', 'better', 'best', 'more', 'most', 'above', 'record', 'high', 'higher', 'highest',
                      'greater', 'greatest', 'larger', 'largest', 'grow', 'grows', 'growing', 'grew', 'grown', 'growth',
                      'expand', 'expands', 'expanding', 'expanded', 'expansion', 'exceed', 'exceeds', 'exceeded',
                      'exceeding', 'beat', 'beats', 'beating'] 

In [200]:
# Sentiment analysis using the above negative and positive keywords.
sentbydaypos=defaultdict(int)
sentbydayneg=defaultdict(int)
sentbyday=defaultdict(int)
for index,row in WSB_reddits.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in Positive:
            sentbydaypos[row['day']] += 1
            sentbyday[row['day']] += 1
        if word in Negative:
            sentbydayneg[row['day']] += 1
            sentbyday[row['day']] =  sentbyday[row['day']] -1


In [208]:
# positive sentiment counts of frequency of positive keywords by day
sentbydaypos

defaultdict(int,
            {Timestamp('2016-01-01 00:00:00'): 23,
             Timestamp('2016-01-02 00:00:00'): 2,
             Timestamp('2016-01-03 00:00:00'): 88,
             Timestamp('2016-01-04 00:00:00'): 28,
             Timestamp('2016-01-05 00:00:00'): 19,
             Timestamp('2016-01-06 00:00:00'): 39,
             Timestamp('2016-01-07 00:00:00'): 79,
             Timestamp('2016-01-08 00:00:00'): 75,
             Timestamp('2016-01-09 00:00:00'): 9,
             Timestamp('2016-01-10 00:00:00'): 17,
             Timestamp('2016-01-11 00:00:00'): 25,
             Timestamp('2016-01-12 00:00:00'): 45,
             Timestamp('2016-01-13 00:00:00'): 32,
             Timestamp('2016-01-14 00:00:00'): 40,
             Timestamp('2016-01-15 00:00:00'): 52,
             Timestamp('2016-01-16 00:00:00'): 49,
             Timestamp('2016-01-17 00:00:00'): 40,
             Timestamp('2016-01-18 00:00:00'): 59,
             Timestamp('2016-01-19 00:00:00'): 64,
             Tim

In [209]:
# negative sentiment counts of frequency of negative keywords by day
sentbydayneg

defaultdict(int,
            {Timestamp('2016-01-01 00:00:00'): 6,
             Timestamp('2016-01-02 00:00:00'): 2,
             Timestamp('2016-01-03 00:00:00'): 58,
             Timestamp('2016-01-04 00:00:00'): 30,
             Timestamp('2016-01-05 00:00:00'): 27,
             Timestamp('2016-01-06 00:00:00'): 62,
             Timestamp('2016-01-07 00:00:00'): 84,
             Timestamp('2016-01-08 00:00:00'): 76,
             Timestamp('2016-01-09 00:00:00'): 18,
             Timestamp('2016-01-10 00:00:00'): 2,
             Timestamp('2016-01-11 00:00:00'): 26,
             Timestamp('2016-01-12 00:00:00'): 43,
             Timestamp('2016-01-13 00:00:00'): 24,
             Timestamp('2016-01-14 00:00:00'): 32,
             Timestamp('2016-01-15 00:00:00'): 75,
             Timestamp('2016-01-16 00:00:00'): 32,
             Timestamp('2016-01-17 00:00:00'): 26,
             Timestamp('2016-01-18 00:00:00'): 76,
             Timestamp('2016-01-19 00:00:00'): 52,
             Time

In [210]:
# (positive - negative) sentiment counts of frequency of (positive - negative) keywords by day
sentbyday

defaultdict(int,
            {Timestamp('2016-01-01 00:00:00'): 17,
             Timestamp('2016-01-02 00:00:00'): 0,
             Timestamp('2016-01-03 00:00:00'): 30,
             Timestamp('2016-01-04 00:00:00'): -2,
             Timestamp('2016-01-05 00:00:00'): -8,
             Timestamp('2016-01-06 00:00:00'): -23,
             Timestamp('2016-01-07 00:00:00'): -5,
             Timestamp('2016-01-08 00:00:00'): -1,
             Timestamp('2016-01-09 00:00:00'): -9,
             Timestamp('2016-01-10 00:00:00'): 15,
             Timestamp('2016-01-11 00:00:00'): -1,
             Timestamp('2016-01-12 00:00:00'): 2,
             Timestamp('2016-01-13 00:00:00'): 8,
             Timestamp('2016-01-14 00:00:00'): 8,
             Timestamp('2016-01-15 00:00:00'): -23,
             Timestamp('2016-01-16 00:00:00'): 17,
             Timestamp('2016-01-17 00:00:00'): 14,
             Timestamp('2016-01-18 00:00:00'): -17,
             Timestamp('2016-01-19 00:00:00'): 12,
             Ti

In [211]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by day
wsb_sentbydayposdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
wsb_sentbydayposdf.columns=(['Date','Pos_Count'])
wsb_sentbydayposdf

,Date,Pos_Count
0,2016-01-01,23
1,2016-01-02,2
2,2016-01-03,88
3,2016-01-04,28
4,2016-01-05,19
...,...,...
1916,2021-04-27,1137
1917,2021-04-28,1104
1918,2021-04-29,805
1919,2021-04-30,1187


In [212]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
wsb_sentbydayposdf.to_csv('./Data/wsb_sentbydayposdf_count.csv', index=False)

In [213]:
# Create a dataframe of the negative sentiment counts of frequency of negative keywords by day
wsb_sentbydaynegdf = pd.DataFrame([sentbydayneg]).transpose().reset_index()
wsb_sentbydaynegdf.columns=(['Date','Neg_Count'])
wsb_sentbydaynegdf

,Date,Neg_Count
0,2016-01-01,6
1,2016-01-02,2
2,2016-01-03,58
3,2016-01-04,30
4,2016-01-05,27
...,...,...
1917,2021-04-27,617
1918,2021-04-28,600
1919,2021-04-29,462
1920,2021-04-30,894


In [214]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
wsb_sentbydaynegdf.to_csv('./Data/wsb_sentbydaynegdf_count.csv', index=False)

In [215]:
# Create a dataframe of the (positive - negative) sentiment counts of frequency of (positive - negative) keywords by day
wsb_sentbydaydf = pd.DataFrame([sentbyday]).transpose().reset_index()
wsb_sentbydaydf.columns=(['Date','Sent_Count'])
wsb_sentbydaydf

,Date,Sent_Count
0,2016-01-01,17
1,2016-01-02,0
2,2016-01-03,30
3,2016-01-04,-2
4,2016-01-05,-8
...,...,...
1917,2021-04-27,520
1918,2021-04-28,504
1919,2021-04-29,343
1920,2021-04-30,293


In [216]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
wsb_sentbydaydf.to_csv('./Data/wsb_sentbydaydf_count.csv', index=False)

In [114]:
# Sentiment analysis using the above negative and positive keywords.
sentbyweekpos=defaultdict(int)
sentbyweekneg=defaultdict(int)
sentbyweek=defaultdict(int)
for index,row in WSB_reddits.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in Positive:
            sentbyweekpos[row['week']] += 1
            sentbyweek[row['week']] += 1
        if word in Negative:
            sentbyweekneg[row['week']] += 1
            sentbyweek[row['week']] =  sentbyweek[row['week']] -1

In [115]:
# positive sentiment counts of frequency of positive keywords by week
sentbyweekpos

defaultdict(int,
            {Timestamp('2021-04-26 00:00:00'): 4994,
             Timestamp('2021-04-19 00:00:00'): 4722,
             Timestamp('2021-04-12 00:00:00'): 3801,
             Timestamp('2021-04-05 00:00:00'): 4854,
             Timestamp('2021-03-29 00:00:00'): 6681,
             Timestamp('2021-03-22 00:00:00'): 627,
             Timestamp('2021-03-15 00:00:00'): 5636,
             Timestamp('2021-03-08 00:00:00'): 16874,
             Timestamp('2021-03-01 00:00:00'): 5150,
             Timestamp('2021-02-22 00:00:00'): 5784,
             Timestamp('2021-02-15 00:00:00'): 9797,
             Timestamp('2021-02-08 00:00:00'): 6475,
             Timestamp('2021-02-01 00:00:00'): 11094,
             Timestamp('2021-01-25 00:00:00'): 35655,
             Timestamp('2021-01-18 00:00:00'): 9865,
             Timestamp('2021-01-11 00:00:00'): 6660,
             Timestamp('2021-01-04 00:00:00'): 4030,
             Timestamp('2020-12-28 00:00:00'): 4608,
             Timestamp('202

In [116]:
# negative sentiment counts of frequency of negative keywords by week
sentbyweekneg

defaultdict(int,
            {Timestamp('2021-04-26 00:00:00'): 2983,
             Timestamp('2021-04-19 00:00:00'): 3094,
             Timestamp('2021-04-12 00:00:00'): 2753,
             Timestamp('2021-04-05 00:00:00'): 3262,
             Timestamp('2021-03-29 00:00:00'): 4989,
             Timestamp('2021-03-22 00:00:00'): 456,
             Timestamp('2021-03-15 00:00:00'): 4035,
             Timestamp('2021-03-08 00:00:00'): 12376,
             Timestamp('2021-03-01 00:00:00'): 3531,
             Timestamp('2021-02-22 00:00:00'): 4126,
             Timestamp('2021-02-15 00:00:00'): 7464,
             Timestamp('2021-02-08 00:00:00'): 4783,
             Timestamp('2021-02-01 00:00:00'): 11405,
             Timestamp('2021-01-25 00:00:00'): 36970,
             Timestamp('2021-01-18 00:00:00'): 6610,
             Timestamp('2021-01-11 00:00:00'): 4186,
             Timestamp('2021-01-04 00:00:00'): 2432,
             Timestamp('2020-12-28 00:00:00'): 2949,
             Timestamp('202

In [117]:
# (positive - negative) sentiment counts of frequency of (positive - negative) keywords by week
sentbyweek

defaultdict(int,
            {Timestamp('2021-04-26 00:00:00'): 2011,
             Timestamp('2021-04-19 00:00:00'): 1628,
             Timestamp('2021-04-12 00:00:00'): 1048,
             Timestamp('2021-04-05 00:00:00'): 1592,
             Timestamp('2021-03-29 00:00:00'): 1692,
             Timestamp('2021-03-22 00:00:00'): 171,
             Timestamp('2021-03-15 00:00:00'): 1601,
             Timestamp('2021-03-08 00:00:00'): 4498,
             Timestamp('2021-03-01 00:00:00'): 1619,
             Timestamp('2021-02-22 00:00:00'): 1658,
             Timestamp('2021-02-15 00:00:00'): 2333,
             Timestamp('2021-02-08 00:00:00'): 1692,
             Timestamp('2021-02-01 00:00:00'): -311,
             Timestamp('2021-01-25 00:00:00'): -1315,
             Timestamp('2021-01-18 00:00:00'): 3255,
             Timestamp('2021-01-11 00:00:00'): 2474,
             Timestamp('2021-01-04 00:00:00'): 1598,
             Timestamp('2020-12-28 00:00:00'): 1659,
             Timestamp('2020-

In [118]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by week
wsb_sentbyweekposdf = pd.DataFrame([sentbyweekpos]).transpose().reset_index()
wsb_sentbyweekposdf.columns=(['Week','Pos_Count'])
wsb_sentbyweekposdf

,Week,Pos_Count
0,2021-04-26,4994
1,2021-04-19,4722
2,2021-04-12,3801
3,2021-04-05,4854
4,2021-03-29,6681
...,...,...
274,2016-01-25,272
275,2016-01-18,423
276,2016-01-11,283
277,2016-01-04,266


In [119]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
wsb_sentbyweekposdf.to_csv('./Data/wsb_sentbyweekposdf_count.csv', index=False)

In [120]:
# Create a dataframe of the negative sentiment counts of frequency of negative keywords by week
wsb_sentbyweeknegdf = pd.DataFrame([sentbyweekneg]).transpose().reset_index()
wsb_sentbyweeknegdf.columns=(['Week','Neg_Count'])
wsb_sentbyweeknegdf

,Week,Neg_Count
0,2021-04-26,2983
1,2021-04-19,3094
2,2021-04-12,2753
3,2021-04-05,3262
4,2021-03-29,4989
...,...,...
274,2016-01-25,240
275,2016-01-18,342
276,2016-01-11,258
277,2016-01-04,299


In [121]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
wsb_sentbyweeknegdf.to_csv('./Data/wsb_sentbyweeknegdf_count.csv', index=False)

In [122]:
# Create a dataframe of the (positive - negative) sentiment counts of frequency of (positive - negative) keywords by week
wsb_sentbyweekdf = pd.DataFrame([sentbyweek]).transpose().reset_index()
wsb_sentbyweekdf.columns=(['Week','Sent_Count'])
wsb_sentbyweekdf

,Week,Sent_Count
0,2021-04-26,2011
1,2021-04-19,1628
2,2021-04-12,1048
3,2021-04-05,1592
4,2021-03-29,1692
...,...,...
274,2016-01-25,32
275,2016-01-18,81
276,2016-01-11,25
277,2016-01-04,-33


In [123]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
wsb_sentbyweekdf.to_csv('./Data/wsb_sentbyweekdf_count.csv', index=False)

In [22]:
wsb_favw = WSB_reddits.loc[:, ["week", "favorites"]]
wsb_favw = wsb_favw.groupby(['week']).sum()
#wsb_favw = wsb_favw.loc[wsb_favw['week']>'2015-12-28']
wsb_favw

,favorites
week,
2015-12-28,221
2016-01-04,2284
2016-01-11,3388
2016-01-18,3258
2016-01-25,2786
...,...
2021-03-29,3100
2021-04-05,2222
2021-04-12,2126


In [23]:
wsb_favw.to_csv('./Data/wsb_favw_count.csv', index=True)

In [25]:
wsb_favd

,day,favorites
27,2016-01-01,2
28,2016-01-01,8
32,2016-01-01,0
33,2016-01-01,1
34,2016-01-01,1
...,...,...
583002,2021-05-01,1
583016,2021-05-01,1
583026,2021-05-01,1
583030,2021-05-01,1


In [26]:
wsb_favd = WSB_reddits.loc[:, ["day", "favorites"]]
wsb_favd = wsb_favd.groupby(['day']).sum()
wsb_favd.rename(columns={"day": "date", "favorites": "favorites"}, inplace=True)
wsb_favd

,favorites
day,
2016-01-01,25
2016-01-02,37
2016-01-03,159
2016-01-04,224
2016-01-05,156
...,...
2021-04-27,558
2021-04-28,452
2021-04-29,455


In [27]:
wsb_favd.to_csv('./Data/wsb_favd_count.csv', index=True)